In [1]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#load pcd file, filter, downsample
#pcdn = o3d.io.read_point_cloud("loop.pcd")
pcdn = o3d.io.read_point_cloud("final_cropped_ground_align.pcd")
pcdn.estimate_normals()
cl, ind = pcdn.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=0.8)

pcd = pcdn.select_by_index(ind)
pcd = pcd.voxel_down_sample(voxel_size=0.1)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(40)
o3d.visualization.draw_geometries([pcd])

In [3]:
def filter_by_normal(pcd):
    ind = []
    for i in range(len(pcd.points)):
        if ((np.abs(pcd.normals[i][2]) < 0.3) and ((np.abs(pcd.normals[i][0]) > 0.8) or (np.abs(pcd.normals[i][1]) > 0.8))):
            pcd.points[i][2] = np.random.rand()*0.01
            pass
        else:
            ind.append(i)
    result = pcd.select_by_index(ind, invert=True)
    return result

In [4]:
def compute_transform(vector1, vector2):
    # Normalize the vectors to unit length
    vector1 = vector1 / np.linalg.norm(vector1)
    vector2 = vector2 / np.linalg.norm(vector2)

    # Compute the rotation matrix
    rotation_matrix = Rotation.align_vectors([vector1], [vector2])[0].as_matrix()

    # Compute the translation vector
    translation_vector = vector2 - np.dot(rotation_matrix, vector1)

    return translation_vector, rotation_matrix

In [5]:
def create_arrow_from_vector(vector, location):
    mesh_normal = vector
    mesh_center = location
    
    #print("Norm average:")
    #print(norm_average)
    
    arrow = o3d.geometry.TriangleMesh.create_cone(0.5,1)
    
    arrow.paint_uniform_color([1,0,0])
    arrow_bb = arrow.get_oriented_bounding_box()
    arrow.compute_triangle_normals()
    arrow.compute_vertex_normals()
    arrow_normal = np.asarray(arrow.vertex_normals)[0]

    translation, rotation = compute_transform(mesh_normal, arrow_normal)  
    
    arrow.rotate(rotation)
    
    arrow.translate(mesh_center, False)
    
    
   
    return arrow
    
    

In [6]:
def are_vectors_perpendicular(v1, v2, threshold):
    
    # Normalize the vectors to unit length
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)
    angle = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    angle = np.degrees(angle)
    
    if (180 <= angle < 360):
        angle -= 180
        
    if ((90-threshold) <= angle <= (90+threshold)):
        return True
    else:
        return False


In [7]:
filtered_pcd = filter_by_normal(pcd)
o3d.visualization.draw_geometries([filtered_pcd])

In [8]:
oboxes = filtered_pcd.detect_planar_patches(
normal_variance_threshold_deg=50,
coplanarity_deg=85,
outlier_ratio=0.75,
min_plane_edge_length=2,
min_num_points=10,
search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50))

print("Detected {} patches".format(len(oboxes)))

geometries = []
meshes = []
for obox in oboxes:

    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
    mesh.paint_uniform_color(obox.color)
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()
    normals = np.asarray(mesh.vertex_normals)

    
    meshes.append(mesh)
    geometries.append(obox)

#o3d.visualization.draw_geometries(geometries + [boundarys.to_legacy()])
o3d.visualization.draw_geometries(meshes+[filtered_pcd])
#for obox, mesh in zip(oboxes, meshes):
 #   print(obox.extent)
  #  print(mesh.vertex_normals[0])

Detected 23 patches


In [9]:
def devide_meshes_hor_ver(meshes):
    
    first = meshes[0]
    tmp = False
    ver_patches = []
    hor_patches = []
    if (are_vectors_perpendicular(first.vertex_normals[0], np.asarray([1,0,0]), 15)):
        ver_patches.append(first)
    else:
        hor_patches.append(first)
        tmp = True
    

    for i in range(1,len(meshes)):
        patch_normal = meshes[i].vertex_normals[0]

        if (are_vectors_perpendicular(first.vertex_normals[0], patch_normal, 10)):
            if (tmp):
                
                ver_patches.append(meshes[i])
            else:
                hor_patches.append(meshes[i])
        else:
            if (tmp):
                hor_patches.append(meshes[i])
            else:
                ver_patches.append(meshes[i])



    return hor_patches, ver_patches


In [10]:
hor_patches, ver_patches = devide_meshes_hor_ver(meshes)
o3d.visualization.draw_geometries(ver_patches)
o3d.visualization.draw_geometries(hor_patches)
print(len(hor_patches), len(ver_patches))

4 19


In [11]:
def get_mesh_distance(mesh1, mesh2, orientation):
    o = {
  "vertical": 0,
  "horizontal": 1}
    bb1 = mesh1.get_oriented_bounding_box()
    bb1_center = bb1.get_center()[1-o[orientation]]
    bb2 = mesh2.get_oriented_bounding_box()
    bb2_center = bb2.get_center()[1-o[orientation]]
    
    
    if (bb1_center < 0 < bb2_center):
        dist = -bb1_center + bb2_center
    elif (bb2_center < 0 < bb1_center):
        dist = bb1_center - bb2_center
    else:
        dist = np.abs(bb1_center-bb2_center)
        
    return dist

In [12]:
def find_nearest_mesh(mesh1, meshes, orientation):
    o = {
  "vertical": 0,
  "horizontal": 1}
    dist = np.Inf
    index = 0
    
    bb1 = mesh1.get_oriented_bounding_box()
    bb1_center = bb1.get_center()[1-o[orientation]]
    
    for mesh2, i in zip(meshes, range(len(meshes))):
        
        bb2 = mesh2.get_oriented_bounding_box()
        bb2_center = bb2.get_center()[1-o[orientation]]
        
        if not (mesh_correspondance2(mesh1, mesh2, orientation)):
            continue
        
        
        if (bb1_center < 0 < bb2_center):
            dist_tmp = -bb1_center + bb2_center
        elif (bb2_center < 0 < bb1_center):
            dist_tmp = bb1_center - bb2_center
        else:
            dist_tmp = np.abs(bb1_center-bb2_center)

        if dist_tmp == 0:
            continue
        if dist_tmp < dist:
            dist = dist_tmp
            index = i
    
    return dist, index
    
    

In [13]:
def mesh_correspondance2(mesh1, mesh2, orientation):
    o = {
  "vertical": 0,
  "horizontal": 1}
    
    bb1 = mesh1.get_oriented_bounding_box()
    bb2 = mesh2.get_oriented_bounding_box()
    bb1_min = bb1.get_min_bound()
    bb1_max = bb1.get_max_bound()
    bb2_min = bb2.get_min_bound()
    bb2_max = bb2.get_max_bound()
    bb1_center = bb1.get_center()[o[orientation]]
    bb2_center = bb2.get_center()[o[orientation]]
    
    bb1_center2 = bb1.get_center()[1-o[orientation]]
    bb2_center2 = bb2.get_center()[1-o[orientation]]
    
    if (bb1_center2 < 0 < bb2_center2):
        dist = -bb1_center2 + bb2_center2
    elif (bb2_center2 < 0 < bb1_center2):
        dist = bb1_center2 - bb2_center2
    else:
        dist = np.abs(bb1_center2-bb2_center2)
    
    if (dist < 1):
        return False
    #print("dist between planes: ", np.abs(bb1_center-bb2_center))
    if (bb1_min[o[orientation]] < bb2_center < bb1_max[o[orientation]]):
        return True
    if (bb2_min[o[orientation]] < bb1_center < bb2_max[o[orientation]]):

        return True
    
    
    return False
    

In [14]:
corr = []
corr_tuples = []
    
for i in range(len(hor_patches)):
    mesh1 = hor_patches[i]
    for j in range(i+1, len(hor_patches)):
        mesh2 = hor_patches[j]
        nearest_dist, _  = find_nearest_mesh(mesh1, hor_patches, "horizontal")
        
      
        if (nearest_dist < get_mesh_distance(mesh1, mesh2, "horizontal")):
            continue
        
        if (mesh_correspondance2(mesh1, mesh2, "horizontal")):
            color = np.random.rand(3)
            mesh1.paint_uniform_color(color)
            mesh2.paint_uniform_color(color)
            #part1 = mesh1.sample_points_poisson_disk(number_of_points=2000)
            #part2 = mesh2.sample_points_poisson_disk(number_of_points=2000)
#
            #n_points1 = len(part1.points)
            #n_points2 = len(part2.points)
            #line_points1 = [np.random.randint(n_points1) for i in range(0,n_points1,10)]
            #line_points2 = [np.random.randint(n_points2) for i in range(0,n_points2,10)]
            #lineset1 = o3d.geometry.LineSet.create_from_point_cloud_correspondences(part1, part2, list(zip(line_points1, line_points2)))
            #lineset1.paint_uniform_color(np.random.rand(3))
            corr.append(mesh1)
            corr.append(mesh2)
            corr_tuples.append((mesh1, mesh2))
o3d.visualization.draw_geometries(corr)

In [15]:

    
for i in range(len(ver_patches)):
    mesh1 = ver_patches[i]
    for j in range(i+1, len(ver_patches)):
        mesh2 = ver_patches[j]
        
        nearest_dist, _  = find_nearest_mesh(mesh1, ver_patches, "vertical")
        
        
        if (nearest_dist < get_mesh_distance(mesh1, mesh2, "vertical")):
            
            continue
            
        
        #print("nearest: ", nearest_dist)
        #print("other: ", get_mesh_distance(mesh1, mesh2, "vertical"))
        if (mesh_correspondance2(mesh1, mesh2, "vertical")):
            #print(mesh1.get_oriented_bounding_box().center)
            #print(mesh2.get_oriented_bounding_box().center)
            #part1 = mesh1.sample_points_poisson_disk(number_of_points=2000)
            #part2 = mesh2.sample_points_poisson_disk(number_of_points=2000)
#
            #n_points1 = len(part1.points)
            #n_points2 = len(part2.points)
            #line_points1 = [np.random.randint(n_points1) for i in range(0,n_points1,10)]
            #line_points2 = [np.random.randint(n_points2) for i in range(0,n_points2,10)]
            #lineset1 = o3d.geometry.LineSet.create_from_point_cloud_correspondences(part1, part2, list(zip(line_points1, line_points2)))
            #lineset1.paint_uniform_color(np.random.rand(3))
            
            color = np.random.rand(3)
            mesh1.paint_uniform_color(color)
            mesh2.paint_uniform_color(color)
            corr.append(mesh1)
            corr.append(mesh2)
            corr_tuples.append((mesh1, mesh2))

o3d.visualization.draw_geometries(corr)

In [16]:
def create_box_at_point(point):
    
    box = o3d.geometry.TriangleMesh.create_box(0.1,0.1,0.1)
    box.paint_uniform_color([1,0,0])
    
    box.translate(point, False)
    
    
    return box

In [17]:
def find_midpoint_between_planes(plane1, plane2):
    center1 = plane1.get_center()
    center2 = plane2.get_center()

    vec_1_2 = center2 - center1

    midpoint = center1 + vec_1_2 / 2
    return midpoint

In [18]:
midpoints = []
marker_meshes = []
for tup in corr_tuples:
    midpoint = find_midpoint_between_planes(tup[0], tup[1])
    
    dist = np.inf
    for point in midpoints:
        tmp = np.linalg.norm(midpoint-point)
        if tmp < dist:
            dist = tmp 
    print(dist)
    if dist > 1.5:
        midpoints.append(midpoint)
        marker_meshes.append(create_box_at_point(midpoint))

print(midpoints)
o3d.visualization.draw_geometries(corr + marker_meshes)

inf
35.87475278967261
16.63422751621609
8.180258375710352
3.771275086543736
9.372809731631573
1.1537782698212358
7.620585322916532
[array([-7.07490385e+00,  5.34960197e+00,  4.76938495e-03]), array([-4.26459698e+01,  6.91581854e-01,  5.39634946e-03]), array([-2.29134539e+01,  2.66506793e-01,  5.35370956e-03]), array([-1.47352763e+01,  8.20111010e-02,  5.12332377e-03]), array([-1.09649626e+01, -3.13434975e-03,  5.02064447e-03]), array([-3.39564254e+01, -2.82144776e+00,  3.89296359e-03]), array([-1.84188814, -0.19015137,  0.00533078])]
